## Overview

This notebook will show you one of the many ways to import a data set into Databricks,  interact with that data and run a ML model on it. When  I say many ways, the beauty of Databricks is you can manipulate the data in a language of your choice and switch on the fly. This notebook assumes that you have a file already inside of DataBricks FileSystem that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

A little about the dataset: The data represents 303 participants, each row representing one participant and 13 observations and 1 label which is binary in  nature. Since this is a labeled data with an outcome of a category, we deem it supervised classification problem. The data was clean, didnt need any preprocessing nor any encoding. All values were numerical.

> #### Column description 

>age: The person's age

>sex : 1 = male, 0 = female.

>cp : the chest pain experience 

>trestbps: The person's resting blood pressure

>chol : the person's cholesterol measurement in mg/dl

>fbs: the person's fasting blood sugar (>120mg/dl, 1 = true, 0 =false)

>restecg : resting resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probale or define left ventricular hypertrophy by Estes' criteria)

>thalach : the person's maximum heart rate achieved

>exag: Exercise induced angina ( 1= yes, 0 = no)

>oldpeak: ST depression induced by exercise relative to rest ('ST' related to positions on the ECG plot)

>slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3:downsloping)

>ca: The number of major vessels (0-3)

>thal: A blood disorder called thalassemia (3= normal, 6= fixed, 7 =reversable defeat)

>##### target: Heart disease (0=no, 1= yes), the target actually stipulates inflamation of the coronary arteries which lead to heart disease.

### What are the factors that contribute to heart disease. In particular inflamation of the Coronary artery.

#### Import mlflow libraries, can be done in cluster aswell for all notebooks that feed off that cluster which need those libraries.

In [5]:
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()
import mlflow


Sat Jun 20 14:09:04 2020 py4j imported

##### Importing dataset and transforming it into a spark data frame

In [7]:
# File location and type
file_location = "dbfs:/FileStore/tables/heart_disease.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


##### Setup up a temporary or permenent view for queries with SQL.

Temporary view (only accessable to this notebook)

In [10]:
# Create a view or table

temp_table_name = "HeartDisease_csv"

df.createOrReplaceTempView(temp_table_name)

In [11]:
%sql

/* Query the created temp table in a SQL cell */

select * from `HeartDisease_csv`

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


Permenant view accessable outside the scope of this notebook via a write to a parquet file

In [13]:
# permanent_table_name = "heartdisease_csv"
# df.write.format("parquet").saveAsTable(permanent_table_name)

Dataset length of records

In [15]:
print("Our dataset has %d rows." % df.count())

Our dataset has 303 rows.

##### Transform spark data frame into a pandas dataframe to run some EDA

In [17]:
import pandas as pd
dt = df.toPandas()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

#### Extract DataFrame correlations

In [19]:
dt.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trestbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalach,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exang,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


#### Visualize correlation matrix using seaborn

High correlation between thal (a blood disorder called thalassemia), oldpeak (ElectroCardioGraphy ECG/EKG), thalach (participants maximum heart rate archieved) and the target (disease or not diseased)

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,10))
corr = dt.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(50, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)

##### Some stats on the features average, minimum and maximum of factors

In [24]:
display(df.describe())

summary,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303,303,303,303,303,303,303,303,303,303,303,303,303,303
mean,54.366336633663366,0.6831683168316832,0.966996699669967,131.62376237623764,246.26402640264027,0.1485148514851485,0.528052805280528,149.64686468646866,0.32673267326732675,1.0396039603960396,1.3993399339933994,0.7293729372937293,2.3135313531353137,0.5445544554455446
stddev,9.08210098983786,0.46601082333962385,1.0320524894832983,17.5381428135171,51.83075098793005,0.35619787492797644,0.525859596359298,22.90516111491409,0.46979446452231655,1.1610750220686346,0.6162261453459622,1.0226063649693276,0.6122765072781408,0.4988347841643915
min,29,0,0,94,126,0,0,71,0,0.0,0,0,0,0
max,77,1,3,200,564,1,2,202,1,6.2,2,4,3,1


##### Stats on the label (target), especially if this is an imbalanced dataset.

In [26]:
%sql

select target from `HeartDisease_csv` count

target
1
1
1
1
1
1
1
1
1
1


##### Split the dataset randomly into 70% for training and 30% for testing.

In [28]:
train, test = df.randomSplit([0.7, 0.3], seed = 0)
(train.count(), test.count())
print("We have %d training examples and %d test examples." % (train.count(), test.count()))



We have 223 training examples and 80 test examples.

Cache the data for speed (data set is really small so this is really not needed)

In [30]:
train.cache()
test.cache()


Out[9]: DataFrame[age: int, sex: int, cp: int, trestbps: int, chol: int, fbs: int, restecg: int, thalach: int, exang: int, oldpeak: double, slope: int, ca: int, thal: int, target: int]

In [31]:
display(train)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
35,1,1,122,192,0,1,174,0,0.0,2,0,2,1
37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
38,1,2,138,175,0,1,173,0,0.0,2,4,2,1
38,1,2,138,175,0,1,173,0,0.0,2,4,2,1
38,1,3,120,231,0,1,182,1,3.8,1,0,3,0
39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
39,0,2,138,220,0,1,152,0,0.0,1,0,2,1
39,1,0,118,219,0,1,140,0,1.2,1,0,3,0


In [32]:
display(test)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
35,1,0,120,198,0,1,130,1,1.6,1,0,3,0
35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
37,0,2,120,215,0,1,170,0,0.0,2,0,2,1
40,1,3,140,199,0,1,178,1,1.4,2,0,3,1
41,0,2,112,268,0,0,172,1,0.0,2,0,2,1
41,1,0,110,172,0,0,158,0,0.0,2,0,3,0
41,1,1,135,203,0,1,132,0,0.0,1,0,1,1
41,1,2,112,250,0,1,179,0,0.0,2,0,2,1


### Data visualization

##### Older people are more likely than younger people to suffer from Heart disease.

In [35]:
display(train.select("target", "age"))

target,age
1,34
1,34
1,35
1,37
1,38
1,38
0,38
1,39
1,39
0,39


##### No real correllation by sex

In [37]:
display(train.select("sex", "target"))
#sex: The person's sex (1 = male, 0 = female)

sex,target
0,1
1,1
1,1
1,1
1,1
1,1
1,0
0,1
0,1
1,0


##### High blood pressure is a high risk factor for heart condition

In [39]:
display(train.select("target", "trestbps"))

target,trestbps
1,118
1,118
1,122
1,130
1,138
1,138
0,120
1,94
1,138
0,118


##### Going higher than your maximum heart rate for long periods of time could be a risk factor for heart condition

In [41]:
display(train.select("target", "thalach"))

target,thalach
1,192
1,174
1,174
1,187
1,173
1,173
0,182
1,179
1,152
0,140


##### When there is high cholesterol in your blood, it clogs the coronary arteries in the form of cholesterol plaque beoming a high risk factor

In [43]:
display(train.select("target", "chol"))

target,chol
1,210
1,182
1,192
1,250
1,175
1,175
0,231
1,199
1,220
0,219


##### Deeper and more widespread ST depression (ECG/EKG) generally indicates more severe or extensive disease.

In [45]:
display(train.select("target","oldpeak"))

target,oldpeak
1,0.7
1,0.0
1,0.0
1,3.5
1,0.0
1,0.0
0,3.8
1,0.0
1,0.0
0,1.2


##### Thalassemia (thal) is a blood disorder that causes heart disease

In [47]:
display(train.select("target","thal"))

target,thal
1,2
1,2
1,2
1,2
1,2
1,2
0,3
1,2
1,2
0,3


#### Separate features from label

In [49]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns
featuresCols.remove('target')
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

Gradient boosting Tree

In [51]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="target", featuresCol="features")

Decision Tree

In [53]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="target", featuresCol="features", maxDepth=3)

#### Cross validation, Grid Search and Evaluators(AUC ROC)

Decision Tree

In [56]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol=dt.getLabelCol())
cv = CrossValidator(estimator=dt, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=5)
paramGrid = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [2])\
  .build()

Gradient Boosting Tree

In [58]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2])\
  .addGrid(gbt.maxIter, [10])\
  .build()
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol=dt.getLabelCol())
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=5)


#### Stages and Pipleline

In [60]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])


#### Train model

In [62]:
pipelineModel = pipeline.fit(train)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

#### Predictions

In [64]:
predictions = pipelineModel.transform(test)

In [65]:
display(predictions.select("target", "prediction", *featuresCols))

target,prediction,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
1,1.0,29,1,1,130,204,0,0,202,0,0.0,2,0,2
1,1.0,35,0,0,138,183,0,1,182,0,1.4,2,0,2
0,0.0,35,1,0,120,198,0,1,130,1,1.6,1,0,3
0,0.0,35,1,0,126,282,0,0,156,1,0.0,2,0,3
1,1.0,37,0,2,120,215,0,1,170,0,0.0,2,0,2
1,1.0,40,1,3,140,199,0,1,178,1,1.4,2,0,3
1,1.0,41,0,2,112,268,0,0,172,1,0.0,2,0,2
0,1.0,41,1,0,110,172,0,0,158,0,0.0,2,0,3
1,1.0,41,1,1,135,203,0,1,132,0,0.0,1,0,1
1,1.0,41,1,2,112,250,0,1,179,0,0.0,2,0,2


In [66]:
predictions.columns

Out[48]: ['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [67]:
selected = predictions.select('target', "prediction", "probability", "age", "chol", "thalach", "thal", "oldpeak", 'trestbps')
display(selected)



target,prediction,probability,age,chol,thalach,thal,oldpeak,trestbps
1,1.0,"List(1, 2, List(), List(0.2231404958677686, 0.7768595041322314))",29,204,202,2,0.0,130
1,1.0,"List(1, 2, List(), List(0.2231404958677686, 0.7768595041322314))",35,183,182,2,1.4,138
0,0.0,"List(1, 2, List(), List(0.7843137254901961, 0.21568627450980393))",35,198,130,3,1.6,120
0,0.0,"List(1, 2, List(), List(0.7843137254901961, 0.21568627450980393))",35,282,156,3,0.0,126
1,1.0,"List(1, 2, List(), List(0.2231404958677686, 0.7768595041322314))",37,215,170,2,0.0,120
1,0.0,"List(1, 2, List(), List(0.7843137254901961, 0.21568627450980393))",40,199,178,3,1.4,140
1,1.0,"List(1, 2, List(), List(0.2231404958677686, 0.7768595041322314))",41,268,172,2,0.0,112
0,0.0,"List(1, 2, List(), List(0.7843137254901961, 0.21568627450980393))",41,172,158,3,0.0,110
1,0.0,"List(1, 2, List(), List(0.7843137254901961, 0.21568627450980393))",41,203,132,1,0.0,135
1,1.0,"List(1, 2, List(), List(0.2231404958677686, 0.7768595041322314))",41,250,179,2,0.0,112


#### Evaluation metrics : AUC ROC

In [69]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [70]:
# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol= 'target')
BinaryClassificationEvaluator()

Out[50]: BinaryClassificationEvaluator_af84d8eef4b9

In [71]:
evaluator.evaluate(predictions)

Out[51]: 0.8344305464121132

###Training and Making Predictions from sklearn

### The x contains all the columsn from the dataset except the 'target' Columns. The y varaible contains the value from the 'target' Columns

In [74]:
# Separating features(X) and target(y)
X = dt.drop('target', axis=1)

In [75]:
print(X)

age sex cp trestbps chol ... exang oldpeak slope ca thal
0 63 1 3 145 233 ... 0 2.3 0 0 1
1 37 1 2 130 250 ... 0 3.5 0 0 2
2 41 0 1 130 204 ... 0 1.4 2 0 2
3 56 1 1 120 236 ... 0 0.8 2 0 2
4 57 0 0 120 354 ... 1 0.6 2 0 2
5 57 1 0 140 192 ... 0 0.4 1 0 1
6 56 0 1 140 294 ... 0 1.3 1 0 2
7 44 1 1 120 263 ... 0 0.0 2 0 3
8 52 1 2 172 199 ... 0 0.5 2 0 3
9 57 1 2 150 168 ... 0 1.6 2 0 2
10 54 1 0 140 239 ... 0 1.2 2 0 2
11 48 0 2 130 275 ... 0 0.2 2 0 2
12 49 1 1 130 266 ... 0 0.6 2 0 2
13 64 1 3 110 211 ... 1 1.8 1 0 2
14 58 0 3 150 283 ... 0 1.0 2 0 2
15 50 0 2 120 219 ... 0 1.6 1 0 2
16 58 0 2 120 340 ... 0 0.0 2 0 2
17 66 0 3 150 226 ... 0 2.6 0 0 2
18 43 1 0 150 247 ... 0 1.5 2 0 2
19 69 0 3 140 239 ... 0 1.8 2 2 2
20 59 1 0 135 234 ... 0 0.5 1 0 3
21 44 1 2 130 233 ... 1 0.4 2 0 2
22 42 1 0 140 226 ... 0 0.0 2 0 2
23 61 1 2 150 243 ... 1 1.0 1 0 2
24 40 1 3 140 199 ... 1 1.4 2 0 3
25 71 0 1 160 302 ... 0 0.4 2 2 2
26 59 1 2 150 212 ... 0 1.6 2 0 2
27 51 1 2 110 175 ... 0 0.6 2 0 2
28 65 0 2 140 417 ... 0 0.8 2 1 2
29 53 1 2 130 197 ... 0 1.2 0 0 2
.. ... ... .. ... ... ... ... ... ... .. ...
273 58 1 0 100 234 ... 0 0.1 2 1 3
274 47 1 0 110 275 ... 1 1.0 1 1 2
275 52 1 0 125 212 ... 0 1.0 2 2 3
276 58 1 0 146 218 ... 0 2.0 1 1 3
277 57 1 1 124 261 ... 0 0.3 2 0 3
278 58 0 1 136 319 ... 0 0.0 2 2 2
279 61 1 0 138 166 ... 1 3.6 1 1 2
280 42 1 0 136 315 ... 1 1.8 1 0 1
281 52 1 0 128 204 ... 1 1.0 1 0 0
282 59 1 2 126 218 ... 0 2.2 1 1 1
283 40 1 0 152 223 ... 0 0.0 2 0 3
284 61 1 0 140 207 ... 1 1.9 2 1 3
285 46 1 0 140 311 ... 1 1.8 1 2 3
286 59 1 3 134 204 ... 0 0.8 2 2 2
287 57 1 1 154 232 ... 0 0.0 2 1 2
288 57 1 0 110 335 ... 1 3.0 1 1 3
289 55 0 0 128 205 ... 1 2.0 1 1 3
290 61 1 0 148 203 ... 0 0.0 2 1 3
291 58 1 0 114 318 ... 0 4.4 0 3 1
292 58 0 0 170 225 ... 1 2.8 1 2 1
293 67 1 2 152 212 ... 0 0.8 1 0 3
294 44 1 0 120 169 ... 1 2.8 0 0 1
295 63 1 0 140 187 ... 1 4.0 2 2 3
296 63 0 0 124 197 ... 1 0.0 1 0 2
297 59 1 0 164 176 ... 0 1.0 1 2 1
298 57 0 0 140 241 ... 1 0.2 1 0 3
299 45 1 3 110 264 ... 0 1.2 1 0 3
300 68 1 0 144 193 ... 0 3.4 1 2 3
301 57 1 0 130 131 ... 1 1.2 1 1 3
302 57 0 1 130 236 ... 0 0.0 1 1 2

[303 rows x 13 columns]

In [76]:
y = dt['target']
print(y)

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
 ..
273 0
274 0
275 0
276 0
277 0
278 0
279 0
280 0
281 0
282 0
283 0
284 0
285 0
286 0
287 0
288 0
289 0
290 0
291 0
292 0
293 0
294 0
295 0
296 0
297 0
298 0
299 0
300 0
301 0
302 0
Name: target, Length: 303, dtype: int32

### We use split up 35% of the data in to the test set and 65% for training

In [78]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [79]:
print(f"X.shape: {X.shape}, y.shape: {y.shape}")
#original dataset

X.shape: (303, 13), y.shape: (303,)

In [80]:
print(f"X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}")
print(f"X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")
#splited datasets

X_train.shape: (196, 13), y_train.shape: (196,)
X_test.shape: (107, 13), y_test.shape: (107,)

### Use the DecisionTreeClassifier to train the algorithm

In [82]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

Out[39]: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
 max_features=None, max_leaf_nodes=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, presort=False, random_state=None,
 splitter='best')

In [83]:
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0
 0 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0
 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0]

### Evaluating the Algorithm from sklearn

### From the confusion matrix, our alogrithm misclassified only 24 out of 107. This is 77% accuracy

In [86]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[40 11]
 [14 42]]
 precision recall f1-score support

 0 0.74 0.78 0.76 51
 1 0.79 0.75 0.77 56

 micro avg 0.77 0.77 0.77 107
 macro avg 0.77 0.77 0.77 107
weighted avg 0.77 0.77 0.77 107

### Conclusion: It shows that people with heart disease tend to be older, and have higher blood pressure, higher cholesterol levels, deeper and more widespread ST etc., than people without the disease.